In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('./data/DatasetV4.csv')
df['client_id'] = df.index
df.head()

,Client ID,Gender,Year,Trip Type,Order Type,Country,Region,Star Rate,Group Type,Meal Option,...,Adult Tourists,Baby Tourists,Kid Tourists,Birthday,Order,Order Date,Trip Start Date,Price,Booking Depth,client_id
0,4197429,1,3,0,0,0,0,2,0,1,...,0,0,2,672537600.0,4803671,1.612731e+09,1.620086e+09,1718.64,85.136111,0
1,4197517,1,3,0,0,0,5,1,0,2,...,0,0,1,342230400.0,4803915,1.612775e+09,1.614816e+09,948.45,23.622917,1
2,4197719,1,3,0,0,0,1,1,0,2,...,0,0,1,-39052800.0,4806034,1.612814e+09,1.633738e+09,2128.15,242.176389,2
3,4199375,1,3,0,0,0,0,1,0,2,...,2,0,1,-300758400.0,4807039,1.612875e+09,1.614384e+09,993.85,17.468056,3
4,4208819,1,3,0,0,0,3,3,2,5,...,3,0,0,484704000.0,4808190,1.612892e+09,1.622938e+09,1121.02,116.265278,4


In [12]:
df.shape

(162140, 25)

In [20]:
df['N Nights']

0         7
1         6
2         4
3         1
4         4
         ..
162135    3
162136    6
162137    2
162138    1
162139    1
Name: N Nights, Length: 162140, dtype: int64

In [10]:
df.columns, [id, gender, birthday, adult/kid/baby tourists, age, Departure City(?)]

Index(['Client ID', 'Gender', 'Year', 'Trip Type', 'Order Type', 'Country',
       'Region', 'Star Rate', 'Group Type', 'Meal Option', 'Destination City',
       'Departure City', 'Age', 'N Nights', 'N Tourists', 'Adult Tourists',
       'Baby Tourists', 'Kid Tourists', 'Birthday', 'Order', 'Order Date',
       'Trip Start Date', 'Price', 'Booking Depth', 'client_id'],
      dtype='object')

In [6]:
from spotlight.interactions import Interactions
from spotlight.factorization.explicit import ExplicitFactorizationModel
from spotlight.factorization.implicit import ImplicitFactorizationModel
from spotlight.cross_validation import random_train_test_split
from spotlight.evaluation import rmse_score
from spotlight.evaluation import mrr_score

models_dict = {}

In [7]:
data = Interactions(user_ids=df["client_id"].values, item_ids=df["Region"].values)
train, test = random_train_test_split(data)

In [9]:
model = ImplicitFactorizationModel(n_iter=10)
%time model.fit(train, verbose=True)

Epoch 0: loss 0.5529947672957735
Epoch 1: loss 0.22088196153589018
Epoch 2: loss 0.09742961300048367
Epoch 3: loss 0.05666683276626603
Epoch 4: loss 0.04145106170657118
Epoch 5: loss 0.03158898746752763
Epoch 6: loss 0.026342364299043103
Epoch 7: loss 0.02219775906732743
Epoch 8: loss 0.019561283739055902
Epoch 9: loss 0.017021488983834373
CPU times: user 10min 30s, sys: 1min 20s, total: 11min 50s
Wall time: 4min 36s


In [41]:
res = model.predict(user_ids=162130)

In [47]:
mrr = mrr_score(model, test)
mrr

In [70]:
N_ITER = 1
stop_loop_debug = 0

for column_name in df.columns[:-1]:
    stop_loop_debug += 1
    print("Training on {} feature".format(column_name))
    model = ImplicitFactorizationModel(n_iter=N_ITER)
    data = Interactions(user_ids=df["client_id"].values, item_ids=df[column_name].values)
    train, test = random_train_test_split(data)
    model.fit(train, verbose=True)
    models_dict[column_name] = {"model" : model, "train": train, "test": test}
    if stop_loop_debug == 3:
        break

Training on Year feature
Epoch 0: loss 0.7045349026690338
Training on Trip Type feature
Epoch 0: loss 0.45107137489718563
Training on Order Type feature
Epoch 0: loss 0.4367696030958164


In [73]:
model = models_dict["Order Type"]["model"]
train = models_dict["Order Type"]["train"]
test = models_dict["Order Type"]["test"]

In [74]:
mrr = mrr_score(model, test)
mrr

array([1., 1., 1., ..., 1., 1., 1.])

3881